In [21]:
#use environment MRI_Analysis

import os.path as op
from itertools import product
import os
import pandas as pd
import openpyxl
import msoffcrypto
import io
import seaborn as sns
from matplotlib import pyplot as plt
import nibabel as nib

import numpy as np
#from paths import root_dir
from paths_funcs import glob_sub_dir



In [22]:
# get coordinates from thesis
root_dir='/media/Data03/Thesis/Dabelstein/derivatives/automated_electrode_extraction'
Name_Bach='Sophie'
#root_dir='/media/Data03/Thesis/Hering/derivatives/automated_electrode_extraction'
#Name_Bach='Kira'
Anode='Anode'
Cathode1='Cathode1'
Cathode2='Cathode2'
Cathode3='Cathode3'

In [23]:
sub_dirs = glob_sub_dir(root_dir)

dict_coord={}

if sub_dirs is None:
    raise FileNotFoundError("No subdirectories found in root directory.")

for sub_dir in sub_dirs:
    dir_array=sub_dir.split(os.sep)
    sub=dir_array[7]
    ses=dir_array[9]
    run=dir_array[10]
    #coords are in voxel space
    coords= np.loadtxt(op.join(sub_dir, "mid.txt"), dtype=int, delimiter=",")
    dict_coord[f'{sub}_{ses}_{run}_{Anode}'] = coords[0]
    dict_coord[f'{sub}_{ses}_{run}_{Cathode1}'] = coords[1]
    dict_coord[f'{sub}_{ses}_{run}_{Cathode2}'] = coords[2]
    dict_coord[f'{sub}_{ses}_{run}_{Cathode3}'] = coords[3]


    #transform voxel in anatomical space
    # Load the Nifti file
    img_root=f'/media/MeMoSLAP_Subjects/derivatives/automated_electrode_extraction/{sub}/unzipped'
    img = nib.load(os.path.join(img_root,f'{sub}_ses-base_acq-mprage_T1w.nii'))

    # Get the affine transformation matrix
    affine = img.affine

    # Transform voxel coordinates into anatomical space
    #voxel_coords = (x, y, z)  # Replace with your voxel coordinates
    dict_coord[f'{sub}_{ses}_{run}_{Anode}'] = np.dot(affine, [coords[0][0],coords[0][1],coords[0][2], 1])[:3]
    dict_coord[f'{sub}_{ses}_{run}_{Cathode1}'] = np.dot(affine, [coords[1][0],coords[1][1],coords[1][2], 1])[:3]
    dict_coord[f'{sub}_{ses}_{run}_{Cathode2}'] = np.dot(affine, [coords[2][0],coords[2][1],coords[2][2], 1])[:3]
    dict_coord[f'{sub}_{ses}_{run}_{Cathode3}'] = np.dot(affine, [coords[3][0],coords[3][1],coords[3][2], 1])[:3]

    #dist = np.zeros(len(coords) ** 2, dtype=int)

    #for idx, (coord1, coord2) in enumerate(product(coords, coords)):
    #    dist[idx] = np.linalg.norm(coord1 - coord2)

    #dist = dist.reshape(len(coords), len(coords))

    #anode = coords[np.argmin(np.sum(dist, axis=1))]

In [24]:
df=pd.DataFrame.from_dict(dict_coord, orient='index', columns=['X','Y','Z'])

df['Electrode'] = np.where(df.index.str.contains(Anode), Anode,
                  np.where(df.index.str.contains(Cathode1), Cathode1,
                  np.where(df.index.str.contains(Cathode2), Cathode2,
                  np.where(df.index.str.contains(Cathode3), Cathode3,'else'))))
df['run'] = np.where(df.index.str.contains('run-01'), 'run-01',
                  np.where(df.index.str.contains('run-02'), 'run-02','else'))
df['Session'] = np.where(df.index.str.contains('ses-1'), 'ses-1',
                  np.where(df.index.str.contains('ses-2'), 'ses-2',
                  np.where(df.index.str.contains('ses-3'), 'ses-3',
                  np.where(df.index.str.contains('ses-4'), 'ses-4','else'))))
df['Subject']=df.index.str.split('_').str[0]
df.to_csv('orginal_coord_sophie.csv')

#melt or unpivot Electrode Dimensions
df = pd.melt(df, id_vars=['Electrode','run','Session','Subject'], value_vars=['X', 'Y', 'Z'], var_name='Dimension', value_name='Coordinates')
df['Experiment']=0

df

,Electrode,run,Session,Subject,Dimension,Coordinates,Experiment
0,Anode,run-01,ses-4,sub-015,X,74.094224,0
1,Cathode1,run-01,ses-4,sub-015,X,74.044583,0
2,Cathode2,run-01,ses-4,sub-015,X,62.659286,0
3,Cathode3,run-01,ses-4,sub-015,X,78.118825,0
4,Anode,run-02,ses-4,sub-015,X,74.062845,0
...,...,...,...,...,...,...,...
2875,Cathode3,run-01,ses-1,sub-038,Z,33.898933,0
2876,Anode,run-02,ses-1,sub-038,Z,60.837214,0
2877,Cathode1,run-02,ses-1,sub-038,Z,76.223275,0
2878,Cathode2,run-02,ses-1,sub-038,Z,57.512562,0


In [25]:
list_sub=[]
for sub in df['Subject'].unique():
    if (len((df[(df['Subject'] == sub)]['Session'].unique())) == 4) & (len((df[(df['Subject'] == sub)]['run'])) == 8*4*3):
        list_sub.append(sub)

print('List of all subjects containing 4 session and 2 runs:',list_sub)
#df_dim_coml=df[df['Subject'].isin(list_sub)]
#df_dim_coml.to_csv('df_dim_compl.csv')


List of all subjects containing 4 session and 2 runs: ['sub-015', 'sub-058', 'sub-011', 'sub-034', 'sub-010', 'sub-062', 'sub-017', 'sub-043', 'sub-025', 'sub-001', 'sub-006']


#

# Add baseline electrodes

In [26]:
dict_Projects={}

#from long to wide format
def unpivot(df):
    #index=df_1.index
    columns=df.columns.to_list()
    df=df[columns[0:13]]
    df=pd.melt(df, id_vars=[columns[0]], value_vars=columns[1:13])
    #df.columns= ["Name","Atmer","Kategorien","Messungen","Werte"]
    
    return(df)


for Project, Name in zip(['P1','P3','P7'],['Mohamed','Harun','Leonardo']):
    
    dict_Projects[f'path_{Project}']=f'/media/AG-Share/01_Studien/00_FOR5429_MeMoSLAP/RU_RegularMeetings/JF_{Name}_{Project}/Coregistrated_Electrodecoordinates/{Project}_Electrode_Position.xlsx'
    dict_Projects[f'df_{Project}']=pd.read_excel(dict_Projects[f'path_{Project}'], header=[0,1,2]).dropna(axis=0)
    print(dict_Projects[f'df_{Project}'].columns)
    dict_Projects[f'df_{Project}']=dict_Projects[f'df_{Project}'].drop(columns=[('Unnamed: 0_level_0', 'Unnamed: 0_level_1',       'Item')])
    
    
    print(dict_Projects[f'df_{Project}'])
    #unpivot data frame
    dict_Projects[f'df_{Project}_unpivot']=unpivot(dict_Projects[f'df_{Project}'])
    dict_Projects[f'df_{Project}_unpivot']['Experiment']=Project
    dict_Projects[f'df_{Project}_unpivot'].columns = ["Subject","Session","Electrode","Dimension",'Coordinates','Experiment']
    dict_Projects[f'df_{Project}_unpivot'] = dict_Projects[f'df_{Project}_unpivot'][dict_Projects[f'df_{Project}_unpivot']['Session'].str.contains('Variability')==False]
    dict_Projects[f'df_{Project}_unpivot']['Session'] = dict_Projects[f'df_{Project}_unpivot']['Session'].str.replace(' Coordinates', '').str.replace('Session ','ses-').str.replace('P9','ses-0')
    dict_Projects[f'df_{Project}_unpivot']['run']='baseline'
    #dict_Projects[f'df_{Project}_unpivot']['Electrode'] = dict_Projects[f'df_{Project}_unpivot']['Electrode'].str.replace('Cathode', 'Cat-')
    dict_Projects[f'df_{Project}_unpivot']['Subject']= dict_Projects[f'df_{Project}_unpivot']['Subject'].str.replace('Sub','sub-') 
    #dict_Projects[f'df_{Project}_unpivot']=dict_Projects[f'df_{Project}_unpivot'].set_index('Subject')
    #dict_Projects[f'df_{Project}_unpivot']['Electrode_Dim']=(dict_Projects[f'df_{Project}_unpivot']['Space'] + '_' +
    #                                                        dict_Projects[f'df_{Project}_unpivot']['Electrode'] + '_' +
    #                                                        dict_Projects[f'df_{Project}_unpivot']['Dimension'])
    #dict_Projects[f'df_{Project}_unpivot']['Exp_Electrode_Dim']=(dict_Projects[f'df_{Project}_unpivot']['Experiment'] + '_' +
    #                                                             dict_Projects[f'df_{Project}_unpivot']['Space'] + '_' +
    #                                                        dict_Projects[f'df_{Project}_unpivot']['Electrode'] + '_' + 
    #                                                        dict_Projects[f'df_{Project}_unpivot']['Dimension'] 
    #                                                        )

MultiIndex([(               'Unnamed: 0_level_0', 'Unnamed: 0_level_1', ...),
            (               'Unnamed: 1_level_0', 'Unnamed: 1_level_1', ...),
            (                   'P9 Coordinates',             'Anode ', ...),
            (                   'P9 Coordinates',             'Anode ', ...),
            (                   'P9 Coordinates',             'Anode ', ...),
            (                   'P9 Coordinates',           'Cathode1', ...),
            (                   'P9 Coordinates',           'Cathode1', ...),
            (                   'P9 Coordinates',           'Cathode1', ...),
            (                   'P9 Coordinates',           'Cathode2', ...),
            (                   'P9 Coordinates',           'Cathode2', ...),
            (                   'P9 Coordinates',           'Cathode2', ...),
            (                   'P9 Coordinates',           'Cathode3', ...),
            (                   'P9 Coordinates',           'Cat

In [27]:
# concatenate ses-base and 
df_concat=pd.concat([df,dict_Projects[f'df_P1_unpivot'],dict_Projects[f'df_P3_unpivot'],dict_Projects[f'df_P7_unpivot']])
df_concat

,Electrode,run,Session,Subject,Dimension,Coordinates,Experiment
0,Anode,run-01,ses-4,sub-015,X,74.094224,0
1,Cathode1,run-01,ses-4,sub-015,X,74.044583,0
2,Cathode2,run-01,ses-4,sub-015,X,62.659286,0
3,Cathode3,run-01,ses-4,sub-015,X,78.118825,0
4,Anode,run-02,ses-4,sub-015,X,74.062845,0
...,...,...,...,...,...,...,...
139,Cathode3,baseline,ses-0,sub-037,Z,68.830000,P7
140,Cathode3,baseline,ses-0,sub-040,Z,38.990000,P7
141,Cathode3,baseline,ses-0,sub-043,Z,60.970000,P7
142,Cathode3,baseline,ses-0,sub-058,Z,62.200000,P7


In [29]:
# add Experiment label for subjects rows in which it's zero
for Project in ['P1','P3','P7']:
    list_subject=df_concat.loc[df_concat['Experiment']==Project,'Subject'].unique()
    df_concat["Experiment"] = np.where(df_concat["Subject"].isin(list_subject), Project, df_concat["Experiment"])
df_concat

,Electrode,run,Session,Subject,Dimension,Coordinates,Experiment
0,Anode,run-01,ses-4,sub-015,X,74.094224,P1
1,Cathode1,run-01,ses-4,sub-015,X,74.044583,P1
2,Cathode2,run-01,ses-4,sub-015,X,62.659286,P1
3,Cathode3,run-01,ses-4,sub-015,X,78.118825,P1
4,Anode,run-02,ses-4,sub-015,X,74.062845,P1
...,...,...,...,...,...,...,...
139,Cathode3,baseline,ses-0,sub-037,Z,68.830000,P7
140,Cathode3,baseline,ses-0,sub-040,Z,38.990000,P7
141,Cathode3,baseline,ses-0,sub-043,Z,60.970000,P7
142,Cathode3,baseline,ses-0,sub-058,Z,62.200000,P7


In [30]:
df_concat.to_csv("df_concat_manually_extr.csv")

# add date

In [166]:
#add baseline coordinates
passwd = 'MeMo_HGW'
path_Recruiting='/media/AG-Share/01_Studien/00_FOR5429_MeMoSLAP/RU_Experiments_Participants/01_Recruiting/MeMoSLAP_ScreeningList.xlsx'
decrypted_workbook = io.BytesIO()
with open(path_Recruiting, 'rb') as file:
    office_file = msoffcrypto.OfficeFile(file)
    office_file.load_key(password=passwd)
    office_file.decrypt(decrypted_workbook)

df_time = pd.read_excel(decrypted_workbook, sheet_name='Subjects')

#path_Recruiting='/media/AG-Share/01_Studien/00_FOR5429_MeMoSLAP/RU_Experiments_Participants/01_Recruiting'


In [167]:
columns= ['Screening ID','Project','DateSession1','DateSession2','DateSession3','DateSession4','DateSession5']
                
df_time=df_time[columns].dropna()
df_time['SubjectID']='not given yet'
df_time              

,Screening ID,Project,DateSession1,DateSession2,DateSession3,DateSession4,DateSession5,SubjectID
0,1,P7,2023-05-24 00:00:00,07.06.2023,2023-10-18,2023-10-25,2023-11-01,not given yet
1,2,P3,2023-08-31 00:00:00,2023-09-13 00:00:00,2023-09-20,2023-10-22,2023-10-29,not given yet
4,22,P7,2023-05-31 00:00:00,2023-06-07 00:00:00,2023-08-23,2023-09-17,2023-09-28,not given yet
8,9,P1,2023-05-17 00:00:00,2023-05-24 00:00:00,2023-08-03,2023-09-27,2023-10-26,not given yet
9,10,P3,2023-06-08 00:00:00,2023-06-15 00:00:00,2023-08-10,2023-11-09,2023-11-22,not given yet
...,...,...,...,...,...,...,...,...
161,161,P7,2024-02-14 00:00:00,2024-02-21 00:00:00,2024-02-28,2024-03-06,2024-03-13,not given yet
178,183,P1,2023-11-26 00:00:00,2023-12-03 00:00:00,2023-12-10,2023-12-17,2024-01-03,not given yet
218,223,P3,2024-01-31 00:00:00,2024-02-07 00:00:00,2024-02-15,2024-02-22,2024-02-29,not given yet
240,245,P3,2024-02-07 00:00:00,2024-02-14 00:00:00,2024-02-21,2024-03-06,2024-03-13,not given yet


In [168]:
passwd='WGH_mEmO'
path_Recruiting='/media/AG-Share/01_Studien/00_FOR5429_MeMoSLAP/RU_Experiments_Participants/01_Recruiting/MeMoSLAP_MatchingList.xlsx'
decrypted_workbook = io.BytesIO()
with open(path_Recruiting, 'rb') as file:
    office_file = msoffcrypto.OfficeFile(file)
    office_file.load_key(password=passwd)
    office_file.decrypt(decrypted_workbook)

df_matching = pd.read_excel(decrypted_workbook, sheet_name='Subjects')

In [169]:
df_matching=df_matching[['SubjectID','Screening ID']].dropna()
dict_df = df_matching.set_index('Screening ID')['SubjectID'].to_dict()
dict_df.keys()

dict_keys([168, 155, 38, 72, 171, 172, 169, 105, 132, 9, 61, 50, 29, 1, 20, 65, 37, 54, 22, 13, 41, 43, 57, 25, 17, 33, 48, 56, 58, 66, 89, 92, 10, 125, 82, 99, 68, 5, 27, 51, 100, 63, 73, 67, 114, 93, 112, 62, 64, 71, 116, 135, 97, 126, 104, 124, 75, 47, 128, 118, 131, 19, 121, 94, 49, 86, 144, 122, 69, 23, 2, 30, 145, 183, 88, 153, 113, 166, 87, 21, 139, 245, 263, 32, 133, 223, 161, 326, 248, 264, 241, 296, 325, 300, 212, 308, 348, 179, 163, 239, 262, 273, 271, 182, 199, 260, 184, 304, 207, 289, 310, 83, 138, 196, 206, 210, 247, 272, 225, 286, 209, 229, 195, 292, 307])

In [170]:
for key, value in dict_df.items():
    df_time.loc[df_time['Screening ID']==key,'SubjectID']=value
df_time.columns=['Screening ID','Project','ses-0','ses-1','ses-2','ses-3','ses-4','Subject']
df_time['Subject']=df_time['Subject'].str.replace('sub','sub-')

In [171]:
df_time_melt=pd.melt(df_time, id_vars=['Subject','Project'],value_vars=['ses-0','ses-1','ses-2','ses-3','ses-4'],var_name='Session', value_name='date')
df_time_melt

,Subject,Project,Session,date
0,sub-005,P7,ses-0,2023-05-24 00:00:00
1,sub-062,P3,ses-0,2023-08-31 00:00:00
2,sub-010,P7,ses-0,2023-05-31 00:00:00
3,sub-001,P1,ses-0,2023-05-17 00:00:00
4,sub-024,P3,ses-0,2023-06-08 00:00:00
...,...,...,...,...
310,sub-078,P7,ses-4,2024-03-13 00:00:00
311,sub-065,P1,ses-4,2024-01-03 00:00:00
312,sub-077,P3,ses-4,2024-02-29 00:00:00
313,sub-073,P3,ses-4,2024-03-13 00:00:00


In [172]:
df_concat['date']='date'
for sub in df_time_melt['Subject'].unique():
    for ses in df_time_melt['Session'].unique():
        date=df_time_melt.loc[(df_time_melt['Subject']==sub)&(df_time_melt['Session']==ses),'date']
        Experiment=df_time_melt.loc[(df_time_melt['Subject']==sub)&(df_time_melt['Session']==ses),'Project']
        df_concat["date"] = np.where((df_concat["Subject"]==sub) & (df_concat["Session"]==ses), date, df_concat["date"])
        df_concat['Experiment'] = np.where((df_concat["Subject"]==sub) & (df_concat["Session"]==ses), Experiment, df_concat["Experiment"])
    

In [173]:
# delete all rows not containing correct date string
df_concat=df_concat[df_concat['date']!='date']
df_concat['date']=pd.to_datetime(df_concat['date']).copy()
df_concat

/tmp/ipykernel_817046/4019678154.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat['date']=pd.to_datetime(df_concat['date']).copy()


,Electrode,run,Session,Subject,Dimension,Coordinates,Experiment,date
0,Anode,run-01,ses-4,sub-015,X,74.094224,P1,2023-10-01
1,Cathode1,run-01,ses-4,sub-015,X,74.044583,P1,2023-10-01
2,Cathode2,run-01,ses-4,sub-015,X,62.659286,P1,2023-10-01
3,Cathode3,run-01,ses-4,sub-015,X,78.118825,P1,2023-10-01
4,Anode,run-02,ses-4,sub-015,X,74.062845,P1,2023-10-01
...,...,...,...,...,...,...,...,...
138,Cathode3,baseline,ses-0,sub-034,Z,61.730000,P7,2023-07-05
139,Cathode3,baseline,ses-0,sub-037,Z,68.830000,P7,2023-07-16
141,Cathode3,baseline,ses-0,sub-043,Z,60.970000,P7,2023-07-30
142,Cathode3,baseline,ses-0,sub-058,Z,62.200000,P7,2023-07-27


In [174]:
# add Experiment label for subjects rows in which it's zero
for Project in ['P1','P3','P7']:
    list_subject=df_concat.loc[df_concat['Experiment']==Project,'Subject'].unique()
    df_concat["Experiment"] = np.where(df_concat["Subject"].copy().isin(list_subject), Project, df_concat["Experiment"])
    
df_concat['Electrode']=df_concat['Electrode'].str.replace('Anode ','Anode')
df_concat['Dimension']=df_concat['Dimension'].str.replace('Y ','Y')

df_concat

/tmp/ipykernel_817046/599912822.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat["Experiment"] = np.where(df_concat["Subject"].copy().isin(list_subject), Project, df_concat["Experiment"])
/tmp/ipykernel_817046/599912822.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat['Electrode']=df_concat['Electrode'].str.replace('Anode ','Anode')
/tmp/ipykernel_817046/599912822.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

,Electrode,run,Session,Subject,Dimension,Coordinates,Experiment,date
0,Anode,run-01,ses-4,sub-015,X,74.094224,P1,2023-10-01
1,Cathode1,run-01,ses-4,sub-015,X,74.044583,P1,2023-10-01
2,Cathode2,run-01,ses-4,sub-015,X,62.659286,P1,2023-10-01
3,Cathode3,run-01,ses-4,sub-015,X,78.118825,P1,2023-10-01
4,Anode,run-02,ses-4,sub-015,X,74.062845,P1,2023-10-01
...,...,...,...,...,...,...,...,...
138,Cathode3,baseline,ses-0,sub-034,Z,61.730000,P7,2023-07-05
139,Cathode3,baseline,ses-0,sub-037,Z,68.830000,P7,2023-07-16
141,Cathode3,baseline,ses-0,sub-043,Z,60.970000,P7,2023-07-30
142,Cathode3,baseline,ses-0,sub-058,Z,62.200000,P7,2023-07-27


In [175]:
# create difference between baseline and sessions

In [176]:
df_diff=df_concat.pivot_table('Coordinates',['Subject','Experiment','Electrode'],['Dimension','Session','run'])
columns=df_diff.columns # baseline columns are [0,9,18,]
for num,column in enumerate(columns):
    if not num in ([0,9,18]):
        if 'X' in column: 
            df_diff[column]=abs(df_diff[column]-df_diff[columns[0]])
        elif 'Y' in column:
            df_diff[column]=abs(df_diff[column]-df_diff[columns[9]])
        elif 'Z' in column:
            df_diff[column]=abs(df_diff[column]-df_diff[columns[18]])
df_diff=df_diff.fillna('No_diff', inplace=False)     
df_diff

Dimension                           X                                          \
Session                         ses-0     ses-1               ses-2             
run                          baseline    run-01    run-02    run-01    run-02   
Subject Experiment Electrode                                                    
sub-001 P1         Anode        74.08  0.200946  1.153867    0.6488  1.641411   
                   Cathode1     64.13  3.481251  3.506729  3.342948  4.295869   
                   Cathode2     66.81  1.774521  2.687752  3.754014  4.613342   
                   Cathode3     80.85  0.339132  0.446937  0.370506  0.384718   
sub-002 P1         Anode       -61.44  4.782833  3.830662  1.029574  1.029574   
...                               ...       ...       ...       ...       ...   
sub-062 P3         Cathode3    -49.44  4.368154  4.368586   7.09035  7.990682   
sub-064 P3         Anode      No_diff   No_diff   No_diff   No_diff   No_diff   
                   Cathode1   No_diff   No_diff   No_diff   No_diff   No_diff   
                   Cathode2   No_diff   No_diff   No_diff   No_diff   No_diff   
                   Cathode3   No_diff   No_diff   No_diff   No_diff   No_diff   

Dimension                                                              \
Session                          ses-3                ses-4             
run                             run-01    run-02     run-01    run-02   
Subject Experiment Electrode                                            
sub-001 P1         Anode      2.427256  3.365964   0.995107  2.373353   
                   Cathode1   0.922848  0.948325   3.790802  1.794316   
                   Cathode2    5.15515  6.068381   3.921451  4.321299   
                   Cathode3   1.408703  1.422916   0.937791  1.297949   
sub-002 P1         Anode      2.059862  2.114547    No_diff   No_diff   
...                                ...       ...        ...       ...   
sub-062 P3         Cathode3   0.765966  0.143241  10.773709  9.854764   
sub-064 P3         Anode       No_diff   No_diff    No_diff   No_diff   
                   Cathode1    No_diff   No_diff    No_diff   No_diff   
                   Cathode2    No_diff   No_diff    No_diff   No_diff   
                   Cathode3    No_diff   No_diff    No_diff   No_diff   

Dimension                           Y  ...                   Z             \
Session                         ses-0  ...      ses-4    ses-0      ses-1   
run                          baseline  ...     run-02 baseline     run-01   
Subject Experiment Electrode           ...                                  
sub-001 P1         Anode       -61.42  ...   1.935457     3.59    1.51913   
                   Cathode1    -81.92  ...  20.809404    22.88  48.661737   
                   Cathode2    -64.15  ...  18.427984   -25.34  55.863048   
                   Cathode3    -34.22  ...   1.836904    13.94   6.176642   
sub-002 P1         Anode       -45.59  ...    No_diff    64.81  27.567409   
...                               ...  ...        ...      ...        ...   
sub-062 P3         Cathode3    -65.27  ...   3.579661    63.26   5.790311   
sub-064 P3         Anode      No_diff  ...    No_diff  No_diff    No_diff   
                   Cathode1   No_diff  ...    No_diff  No_diff    No_diff   
                   Cathode2   No_diff  ...    No_diff  No_diff    No_diff   
                   Cathode3   No_diff  ...    No_diff  No_diff    No_diff   

Dimension                                                                 \
Session                                      ses-2                 ses-3   
run                              run-02     run-01     run-02     run-01   
Subject Experiment Electrode                                               
sub-001 P1         Anode       0.493106   8.702304  10.610953   7.266394   
                   Cathode1   49.359328  59.273652  58.247627  52.787156   
                   Cathode2   55.719649  33.056381   33.98064  41.965899   


In [177]:
#serie_new=df_diff.stack(future_stack=True).stack(future_stack=True).stack(future_stack=True)
df_diff_long=df_diff.stack().stack().stack()
df_diff_long=df_diff_long.to_frame().reset_index().dropna()
df_diff_long.columns=['Subject', 'Experiment', 'Electrode', 'run', 'Session', 'Dimension', 'Coordinate difference in mm']
df_diff_long.loc[df_diff_long['run']=='baseline','Coordinate difference in mm']=0
df_diff_long.dropna()


,Subject,Experiment,Electrode,run,Session,Dimension,Coordinate difference in mm
0,sub-001,P1,Anode,baseline,ses-0,X,0
1,sub-001,P1,Anode,baseline,ses-0,Y,0
2,sub-001,P1,Anode,baseline,ses-0,Z,0
3,sub-001,P1,Anode,run-01,ses-1,X,0.200946
4,sub-001,P1,Anode,run-01,ses-1,Y,0.103142
...,...,...,...,...,...,...,...
4963,sub-064,P3,Cathode3,run-02,ses-3,Y,No_diff
4964,sub-064,P3,Cathode3,run-02,ses-3,Z,No_diff
4965,sub-064,P3,Cathode3,run-02,ses-4,X,No_diff
4966,sub-064,P3,Cathode3,run-02,ses-4,Y,No_diff


In [178]:
df_diff_long['date']='date'
for sub in df_time_melt['Subject'].unique():
    for ses in df_time_melt['Session'].unique():
        date=df_time_melt.loc[(df_time_melt['Subject']==sub)&(df_time_melt['Session']==ses),'date']
        Experiment=df_time_melt.loc[(df_time_melt['Subject']==sub)&(df_time_melt['Session']==ses),'Project']
        df_diff_long["date"] = np.where((df_diff_long["Subject"]==sub) & (df_diff_long["Session"]==ses), date, df_diff_long["date"])
        df_diff_long['Experiment'] = np.where((df_diff_long["Subject"]==sub) & (df_diff_long["Session"]==ses), Experiment, df_diff_long["Experiment"])
df_diff_long=df_diff_long[(df_diff_long['date']!='date') & (df_diff_long['Coordinate difference in mm']!='No_diff')]
df_diff_long['date']=pd.to_datetime(df_diff_long['date'])

In [179]:
df_concat.to_csv(f'concat_{Name_Bach}.csv')
df_diff_long.to_csv(f'df_diff_long_{Name_Bach}.csv')
df_diff_long['date'].unique()

array(['2023-05-17T00:00:00.000000000', '2023-05-24T00:00:00.000000000',
       '2023-08-03T00:00:00.000000000', '2023-09-27T00:00:00.000000000',
       '2023-10-26T00:00:00.000000000', '2023-08-24T00:00:00.000000000',
       '2023-11-29T00:00:00.000000000', '2023-06-07T00:00:00.000000000',
       '2023-08-23T00:00:00.000000000', '2023-10-25T00:00:00.000000000',
       '2023-07-06T00:00:00.000000000', '2023-10-18T00:00:00.000000000',
       '2023-05-25T00:00:00.000000000', '2023-05-31T00:00:00.000000000',
       '2023-10-19T00:00:00.000000000', '2023-09-17T00:00:00.000000000',
       '2023-09-28T00:00:00.000000000', '2023-06-01T00:00:00.000000000',
       '2023-07-05T00:00:00.000000000', '2023-09-07T00:00:00.000000000',
       '2023-09-21T00:00:00.000000000', '2023-06-04T00:00:00.000000000',
       '2024-01-24T00:00:00.000000000', '2024-02-04T00:00:00.000000000',
       '2023-08-27T00:00:00.000000000', '2023-10-29T00:00:00.000000000',
       '2023-11-05T00:00:00.000000000', '2023-06-14